In [3]:
# imports
import sys
import openmm
import openmm.app as app
import openmm.unit as u

import numpy as np
import mdtraj
import torch

import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from rdkit.Geometry import Point3D

from parmed.charmm import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet
from parmed.openmm import StateDataReporter

In [4]:
pdb = app.pdbfile.PDBFile('/home/yppatel/misc/clean_idp_rl/disordered_chignolin/GYDPETGTWG.pdb')

forcefield = app.forcefield.ForceField('amber14/protein.ff14SBonlysc.xml', 'implicit/gbn2.xml')
system = forcefield.createSystem(
    pdb.topology, nonbondedMethod = app.forcefield.NoCutoff, constraints = app.forcefield.HBonds,
)

integrator = openmm.VerletIntegrator(0.002 * u.picoseconds)
platform = openmm.Platform.getPlatformByName("CUDA")

# start at 1, since we assume 0 is being used by PyTorch, to avoid GPU memory issues
assigned_gpu = np.random.randint(1, torch.cuda.device_count())
prop = dict(CudaPrecision="mixed", DeviceIndex=f"{assigned_gpu}")
simulation = app.Simulation(pdb.topology, system, integrator, platform, prop)

In [5]:
def optimize():
    simulation.minimizeEnergy(maxIterations=1000)
    return simulation.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)

In [32]:
pdb = app.pdbfile.PDBFile('chignolin_confs/1uao_autopsf_3.pdb')
simulation.context.setPositions(pdb.positions)
optimize()

Quantity(value=-391.2029892914005, unit=kilocalorie/mole)

In [8]:
def np_to_mm(arr: np.ndarray, unit: openmm.unit=u.angstrom):
    wrapped_val = openmm.unit.quantity.Quantity(arr, unit)
    return wrapped_val

In [12]:
pdb = '/home/yppatel/misc/clean_idp_rl/disordered_chignolin/GYDPETGTWG.pdb'
mol = Chem.rdmolfiles.MolFromPDBFile(pdb, removeHs=False)
AllChem.EmbedMultipleConfs(mol, numConfs=1000, numThreads=-1)

[12:59:29] Molecule does not have explicit Hs. Consider calling AddHs()


In [13]:
energies = []
for i in range(mol.GetNumConformers()):
    pos_conf = np_to_mm(mol.GetConformer(i).GetPositions())
    simulation.context.setPositions(pos_conf)
    energies.append(optimize())

In [14]:
print(sorted(energies))

[Quantity(value=-369.6365444461036, unit=kilocalorie/mole), Quantity(value=-368.8443646626666, unit=kilocalorie/mole), Quantity(value=-368.0788033040905, unit=kilocalorie/mole), Quantity(value=-367.88901936518596, unit=kilocalorie/mole), Quantity(value=-367.8518145061539, unit=kilocalorie/mole), Quantity(value=-367.69986479774843, unit=kilocalorie/mole), Quantity(value=-367.51294711248187, unit=kilocalorie/mole), Quantity(value=-367.46720685570943, unit=kilocalorie/mole), Quantity(value=-367.4503897466097, unit=kilocalorie/mole), Quantity(value=-367.0979233292923, unit=kilocalorie/mole), Quantity(value=-367.0173457327215, unit=kilocalorie/mole), Quantity(value=-366.7831251017275, unit=kilocalorie/mole), Quantity(value=-366.77827323044625, unit=kilocalorie/mole), Quantity(value=-366.7396631834892, unit=kilocalorie/mole), Quantity(value=-366.5119104884725, unit=kilocalorie/mole), Quantity(value=-366.2683555457359, unit=kilocalorie/mole), Quantity(value=-365.7679005069975, unit=kilocalori